In [ ]:
import pandas as pd
import numpy as np
from geopy import distance
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
import matplotlib.pyplot as plt
import seaborn as sns
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size':16})
rc('text', usetex=True)

Create a root node that corresponds to the node on the original dataset that has the highest betweenness centrality measure.

In [ ]:
root_node_coordinates = (34.812600,-118.878712, None)
# The original index of the node that the root node connects to
root_node_index_in_original_dataset = 15457

Read node file. Where small_nodes_df refers to the collection of nodes without degree 2 nodes.

In [ ]:
small_nodes_df = pd.read_csv("data/cal_smoothened_node.txt")
small_nodes_df.rename(columns={"lat": "long", "long": "lat", "Unnamed: 0":"OriginalID"}, inplace=True)
small_nodes_df = small_nodes_df[["lat", "long", "OriginalID"]]

Add root node to adjacency matrix.

In [ ]:
small_nodes_df.loc[len(small_nodes_df)] = root_node_coordinates
small_nodes_df

In [ ]:
root_node_adjacenct_index = small_nodes_df.index[small_nodes_df["OriginalID"] == root_node_index_in_original_dataset][0]
root_node_adjacenct_index

In [ ]:
small_nodes_df.to_pickle("data/processed/node_coordinates.pkl")

Recalculate weights between 2 points after smoothing. The smoothing algorithm that we implemented does account for the sum of weights on graph correctly because it was intended for trees. 

In [ ]:
adjacency_matrix = np.loadtxt("data/cal_smoothened_graph.txt")
adjacency_matrix.shape

Add root node to adjacency matrix.

In [ ]:
adjacency_matrix = np.vstack((adjacency_matrix, np.zeros(len(adjacency_matrix))))
adjacency_matrix = np.hstack((adjacency_matrix, np.zeros((len(adjacency_matrix), 1))))
adjacency_matrix[-1, root_node_adjacenct_index] = 1
adjacency_matrix[root_node_adjacenct_index, -1] = 1
print(adjacency_matrix[-1, :])
print(adjacency_matrix[:, -1])

In [ ]:
small_node_array = small_nodes_df.to_numpy()
for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix)):
        if adjacency_matrix[i, j] > 0:
            point_i = (small_node_array[i, 0], small_node_array[i, 1])
            point_j = (small_node_array[j, 0], small_node_array[j, 1])
            adjacency_matrix[i, j] = distance.distance(point_i, point_j).m
print(adjacency_matrix)

In [ ]:
np.save("data/processed/adjacency_matrix", adjacency_matrix)
np.savetxt("data/processed/adjacency_matrix.txt", adjacency_matrix)

Calculate geodesic distance between any 2 pairs of nodes

In [ ]:
small_node_array = small_nodes_df.to_numpy()
geodesic_matrix = np.zeros((len(small_node_array), len(small_node_array)))
for i in range(len(small_node_array)):
    for j in range(len(small_node_array)):
        point_i = (small_node_array[i, 0], small_node_array[i, 1])
        point_j = (small_node_array[j, 0], small_node_array[j, 1])
        geodesic_matrix[i, j] = distance.distance(point_i, point_j).m

In [ ]:
geodesic_matrix

In [ ]:
np.save("data/processed/geodesic_matrix", geodesic_matrix)
np.savetxt("data/processed/geodesic_matrix.txt", geodesic_matrix)

In [ ]:
adjacency_matrix = np.loadtxt("data/cal_smoothened_adjacency_matrix.txt")
graph = csr_matrix(adjacency_matrix)
dist_matrix = shortest_path(graph, directed=False)
np.save("data/processed/graph_distance", dist_matrix)
np.savetxt("data/processed/graph_distance.txt", dist_matrix)

In [ ]:
adjacency_matrix

In [ ]:
dist_matrix

In [ ]:
def create_scenarios(number_of_scenario, geodesic_matrix):
    groups = [set() for _ in range(number_of_scenario)]
    permutation = np.random.choice(len(geodesic_matrix), number_of_scenario, replace=False)
    for i, center in enumerate(permutation):
        radius = np.random.uniform(5000, 200000)
        eligible = np.where(geodesic_matrix[center, :] < radius)[0]
        farthest = np.max(geodesic_matrix[center, eligible])
        for node in eligible:
            probability_of_picking = 1-(geodesic_matrix[center, node] / farthest)
            if np.random.uniform() -0.2 < probability_of_picking:
                groups[i].add(node)
    return groups

In [ ]:
groups = create_scenarios(50, geodesic_matrix)

In [ ]:
print(np.mean([len(group) for group in groups]))

In [ ]:
scenario_of_node = [[] for _ in range(len(geodesic_matrix))]
for node in range(len(geodesic_matrix)):
    for i, group in enumerate(groups):
        if node in group:
            scenario_of_node[node].append(i)

In [ ]:
print(np.mean([len(x) for x in scenario_of_node]))

In [ ]:
qgis_group_vis_data = []
for group_idx, group in enumerate(groups):
    for item in group:
        qgis_group_vis_data.append([small_node_array[item, 0], small_node_array[item, 1], group_idx])

In [ ]:
qgis_group_vis_data = pd.DataFrame(qgis_group_vis_data, columns=["lat", "lon", "group"])
qgis_group_vis_data.to_csv("data/qgis_group_vis_data.csv")

In [ ]:
qgis_group_vis_data

In [ ]:
a = set()

In [ ]:
def create_sensing_matrix(groups, small_node_df):
    sensing_matrix = np.zeros((len(small_node_df), len(groups)), dtype=np.int8)
    for group_idx, group in enumerate(groups):
        for node in group:
            sensing_matrix[node, group_idx] = 1
    return sensing_matrix

In [ ]:
sensing_matrix = create_sensing_matrix(groups, small_nodes_df)

In [ ]:
sensing_matrix = sensing_matrix[:-1, :]
print(len(sensing_matrix))

In [ ]:
np.savetxt("data/processed/sensing_matrix.txt", sensing_matrix, fmt="%d")
np.save("data/processed/sensing_matrix", sensing_matrix)

In [ ]:
groups

In [ ]:
np.where(sensing_matrix[0, :] == 1)

In [ ]:
np.where(sensing_matrix[:, 7] == 1)

In [ ]:
len(set([tuple(np.where(sensing_matrix[:, i] == 1)[0]) for i in range(len(groups))]))

Convert to pickles for space efficiency

In [ ]:
np.load("data/processed/adjacency_matrix.npy")